# 데이터

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
content = pd.read_csv(f'{DATA_PATH}content_shuffled_df.csv')
shuffled = pd.read_csv(f'{DATA_PATH}shuffled_df.csv')
yogiyo = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


#--------Seq2Seq 단어단위--------

In [18]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=7738b39756e87498e4284109538dfd0a54c7d260a8bc47724d8fc1dc0107102a
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [19]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [20]:
content

,review,owner_reply
0,오늘도맛있게해주셔서감사합니다,"m0님,안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다항상 고객님들 주문 하나하..."
1,너무 맛있어요 감사합니다,저희 매장 찾아주신 것도 감사한데 리뷰까지..감동입니다계속해서 더 많은 기대 부탁드...
2,먹다가 정신이 없어서 2조각 순삭하고 찍었네요,식사는 맛있게 하셨나요? 혜화 대학로 SNS피자랭킹맛집피자보이시나 대학로점을 믿고주...
3,맛있어요. 기존에 시켜먹던데 버리고 .여기서만 시키게요,만족해주신 그 맛! 절대 변하지 않고 앞으로도 쭉 만족하실 수 있도록 최선을 다하겠...
4,주말저녁 맛있게먹었어요,안녕하세요 고객님 D번거로우실텐데 리뷰 작성해 주셔서 감사합니다!앞으로도 맛있게 ...
...,...,...
15195,맛도 너무 좋고 양도 넉넉하고 시간은 겁나 빠르게 맛있습니다. 재주문 나중...,맛있게 드셔주셔서 감사드립니다!!맛도 좋고 양도 좋고 가성비좋고3박자가 다 어우러진...
15196,혼자먹기 많아용 배달은 20분만에 왔어요 잘먹었습니다,황궁 찾아주셔서 넘넘 감사합니다세스코사용으로 청결하고 빠른배달과 친절한서비스로찾아뵙...
15197,항상 맛있게 먹고 있습니다.,안녕하세요.삽교원조두리곱창입니다.저희 음식이 입맛에 맞으셨다니 정말 다행이고 뿌듯합...
15198,맛있어서 먹다가 중간에 찍었어요 3인이서 폭풍흡입중 맛있네요,고돼지 송파점입니다 별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다.위생과 청결...


In [21]:
train = content[:3000]

In [22]:
text = train["review"][0]
text

'오늘도맛있게해주셔서감사합니다'

# 형태소 단위로 토큰화

## 어휘집 생성 (리뷰) - 형태소 단위

In [23]:
result = kiwi.tokenize(train["review"].astype(str))

src_data = []
for tokens in result:
    tokens = [ t.form for t in tokens]
    src_data.append(tokens)

In [24]:
min( len(lst) for lst in src_data)

0

In [25]:
word_count = [len(lst) for lst in src_data]
mask = np.array(word_count) == 1
train.loc[mask]

,review,owner_reply
158,NaN,고돼지 송파점입니다 별점만점 그리고 따뜻한 리뷰 작성해주셔서 감사합니다.위생과 청결...
367,NaN,안녕하세요!!저희 미.친피자가 고객님 입맛에 잘 맞으신거 같아 너무나도 행복한 마...
593,굳,맛있게 드셔 주시고 소중한 리뷰 남겨주셔서 감사합니다 만족스러운 식사 하신 것 같아...
769,NaN,커리 입에 맞으셨나 모르겠습니다 사진보니 포크도 땡기고 커리도 땡기고.. 고민고민신...
782,NaN,"안녕하세요항상 부족함 없는 식사 하실수 있도록초심 잃지않고 좋은재료, 정성담은 음식..."
1018,NaN,저팔계중국마라탕덮밥요리입니다.저희 음식을 맛있게 드시고 소중한 사진리뷰 남겨주신 V...
1113,NaN,오늘도 맛있는 식사 하셨을까요? 저희는 최상의 맛을 위해 신선한 식재료만 사용하고 ...
1293,어쳐처류자러하고갸햏,감사합니다.무슨 뜻인지 모르겠으나 별 다섯개로 만족하겠습니다.더나은 음식으로 보답해...
1553,NaN,자주 시켜주셔서 감사드립니다 신메뉴3종 출시 되었으니 앞으로도 많은 애용 부탁드립니...
1666,NaN,리뷰 감사드립니다 요즘이 최고점 제철인 참숭어를 가장 큰 사이즈를 수급하여 두툼...


In [26]:
from torchtext.vocab import build_vocab_from_iterator
vocab_review = build_vocab_from_iterator(src_data,specials=["<pad>","<unk>"])
vocab_review.set_default_index(vocab_review["<unk>"])
len(vocab_review)

3378

In [27]:
src_data = [ vocab_review(tokens) for tokens in src_data ]
src_data[0]

[71, 10, 3, 11, 6, 13, 39, 31, 15, 46, 6, 24]

In [28]:
vocab_review

Vocab()

## 어휘집 생성 (답변) - 형태소

In [29]:
sos_token = "<sos>"
eos_token = "<eos>"

In [30]:
result = kiwi.tokenize(train["owner_reply"].astype(str))

tgt_data = []
for tokens in result:
    tokens = [sos_token]+ [ t.form for t in tokens] +[eos_token]
    tgt_data.append(tokens)

In [31]:
tgt_data[1]

['<sos>',
 '저희',
 '매장',
 '찾',
 '어',
 '주',
 '시',
 'ᆫ',
 '것',
 '도',
 '감사',
 '하',
 'ᆫ데',
 '리뷰',
 '까지',
 '..',
 '감동',
 '이',
 'ᆸ니다',
 '계속',
 '하',
 '어서',
 '더',
 '많',
 '은',
 '기대',
 '부탁드리',
 '어요',
 '부응',
 '하',
 'ᆯ',
 '수',
 '있',
 '도록',
 '최선',
 '을',
 '다',
 '하',
 '는',
 '모습',
 '을',
 '보이',
 '겠',
 '습니다',
 '다시',
 '찾',
 '어',
 '주',
 '시',
 '어도',
 '5',
 '점',
 '받',
 '을',
 '수',
 '있',
 '게',
 ',',
 '방심',
 '하',
 '지',
 '않',
 '고',
 '노력',
 '하',
 '고',
 '있',
 '겠',
 '습니다',
 '!',
 '<eos>']

In [32]:
vocab_reply = build_vocab_from_iterator(tgt_data,specials=["<pad>","<unk>",sos_token,eos_token])
vocab_reply.set_default_index(vocab_reply["<unk>"])
len(vocab_reply)

3716

In [33]:
vocab_reply

Vocab()

In [34]:
tgt_data = [ vocab_reply(tokens) for tokens in tgt_data ]
tgt_data[0]

[2,
 2421,
 29,
 38,
 61,
 91,
 148,
 5,
 34,
 195,
 4,
 5,
 9,
 6,
 21,
 51,
 11,
 4,
 7,
 41,
 32,
 29,
 33,
 37,
 269,
 20,
 76,
 17,
 124,
 5,
 194,
 4,
 18,
 14,
 15,
 89,
 12,
 33,
 6,
 27,
 42,
 25,
 13,
 411,
 516,
 334,
 4,
 5,
 9,
 35,
 73,
 47,
 11,
 4,
 7,
 94,
 483,
 290,
 175,
 292,
 430,
 338,
 4,
 5,
 16,
 12,
 25,
 521,
 15,
 96,
 90,
 4,
 5,
 9,
 35,
 3]

In [35]:
len(src_data), len(tgt_data)

(3000, 3000)

In [35]:
# 데이터 전처리 (패딩, 길이 확인)
# - 어휘집의 길이보다, 숫자 index의 개수가 많아지면 안됨.
# - 어휘집의 길이 만큼으로 잘라내는 작업 필요함. -> 오류날시 시도해볼 것

# 데이터셋 클래스 만들기

In [36]:
class TranslateDataset(torch.utils.data.Dataset):
    def __init__(self,src,tgt):
        self.src = src
        self.tgt = tgt

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        item = {}
        item["src"] = torch.tensor(self.src[idx])
        item["tgt"] = torch.tensor(self.tgt[idx])

        return item

In [37]:
def collate_fn(samples):
    src = [ item["src"] for item in samples ]
    src = torch.nn.utils.rnn.pad_sequence(src,batch_first=True)
    tgt = [ item["tgt"] for item in samples ]
    tgt = torch.nn.utils.rnn.pad_sequence(tgt,batch_first=True)
    return {"src":src, "tgt":tgt}

In [38]:
dt = TranslateDataset(src_data,tgt_data)
dl = torch.utils.data.DataLoader(dt,batch_size=2, shuffle=False, collate_fn = collate_fn)
batch = next(iter(dl))
batch

{'src': tensor([[71, 10,  3, 11,  6, 13, 39, 31, 15, 46,  6, 24],
         [23,  3,  5, 46,  6, 24,  0,  0,  0,  0,  0,  0]]),
 'tgt': tensor([[   2, 2421,   29,   38,   61,   91,  148,    5,   34,  195,    4,    5,
             9,    6,   21,   51,   11,    4,    7,   41,   32,   29,   33,   37,
           269,   20,   76,   17,  124,    5,  194,    4,   18,   14,   15,   89,
            12,   33,    6,   27,   42,   25,   13,  411,  516,  334,    4,    5,
             9,   35,   73,   47,   11,    4,    7,   94,  483,  290,  175,  292,
           430,  338,    4,    5,   16,   12,   25,  521,   15,   96,   90,    4,
             5,    9,   35,    3],
         [   2,   31,   71,   44,    5,    9,    6,    8,   92,   24,   11,    4,
           355,   34,  118,  202,  375,   10,    7,  381,    4,   21,   50,  130,
            22,  357,  166,   46, 1078,    4,   27,   42,   25,   43,   98,   17,
            68,    4,   13,  371,   17,  198,   18,   14,  135,   44,    5,    9,
           

In [39]:
dl

# 인코더 모델 만들기

In [40]:
class Encoder(torch.nn.Module):
    def __init__(self,num_embeddings, embedding_dim):
        super().__init__()
        self.emb_layer =torch.nn.Embedding(num_embeddings, embedding_dim)
        self.rnn_layer = torch.nn.LSTM(embedding_dim, embedding_dim*2, batch_first=True,bidirectional=True)
    def forward(self,x):
        x = self.emb_layer(x)
        _, (hn,cn) = self.rnn_layer(x)
        return hn,cn

In [41]:
encoder = Encoder(len(vocab_review),64)
hn, cn = encoder(batch["src"])
hn.shape, cn.shape

(torch.Size([2, 2, 128]), torch.Size([2, 2, 128]))

# 디코더 모델 만들기

In [42]:
class Decoder(torch.nn.Module):
    def __init__(self,num_embeddings, embedding_dim):
        super().__init__()
        self.emb_layer =torch.nn.Embedding(num_embeddings, embedding_dim)
        self.rnn_layer = torch.nn.LSTM(embedding_dim, embedding_dim*4, batch_first=True)
        self.fc_layer = torch.nn.Linear(embedding_dim*4, num_embeddings)

    def forward(self,x, hn,cn):
        x = self.emb_layer(x)

        hn = hn.permute(1,0,2).flatten(1).unsqueeze(0)
        cn = cn.permute(1,0,2).flatten(1).unsqueeze(0)
        _, (hn,cn) = self.rnn_layer(x, (hn,cn))

        pred = self.fc_layer(hn[-1]) # n_layer ,batch, feature -> batch, feature
        return pred, hn,cn

In [43]:
hn.permute(1,0,2).flatten(1).unsqueeze(0).shape

torch.Size([1, 2, 256])

In [44]:
cn.permute(1,0,2).flatten(1).unsqueeze(0).shape

torch.Size([1, 2, 256])

In [45]:
tgt = batch["tgt"][:,0].view(-1,1)
tgt

tensor([[2],
        [2]])

In [46]:
decoder = Decoder(len(vocab_reply),64)
pred, hn, cn = decoder(tgt,hn,cn)
pred.shape, hn.shape, cn.shape

(torch.Size([2, 3716]), torch.Size([1, 2, 256]), torch.Size([1, 2, 256]))

# Seq2Seq 모델

In [47]:
class Net(torch.nn.Module):
    def __init__(self,vocab_size_src, vocab_size_tgt,embedding_dim = 64, device ='cpu' ):
        super().__init__()
        self.encoder = Encoder(vocab_size_src,embedding_dim)
        self.decoder = Decoder(vocab_size_tgt,embedding_dim)

        self.vocab_size_tgt = vocab_size_tgt
        self.device = device

    def forward(self,src,tgt,hn=None,cn=None, teacher_forcing_ratio = 0.5):

        batch_size = tgt.shape[0]
        tgt_len = tgt.shape[1]
        pred = torch.zeros(batch_size,tgt_len,self.vocab_size_tgt ).to(self.device)

        if hn is None:
            hn, cn = self.encoder(src)

        dec_input = tgt[:,0].view(-1,1)
        for t in range(1,tgt_len):# t 는 정답 시점
            output, hn,cn = self.decoder(dec_input,hn,cn)

            pred[:,t] = output
            dec_input = output.argmax(1).view(-1,1)

            if random.random() < teacher_forcing_ratio:
                dec_input = tgt[:,t].view(-1,1)

        return pred, hn , cn

In [48]:
len(vocab_review), len(vocab_reply)

(3378, 3716)

In [49]:
model = Net(len(vocab_review),len(vocab_reply))
pred, hn , cn = model(batch["src"],batch["tgt"])

pred.shape, hn.shape, cn.shape

(torch.Size([2, 76, 3716]), torch.Size([1, 2, 256]), torch.Size([1, 2, 256]))

In [50]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        src = batch["src"].to(device)
        tgt = batch["tgt"].to(device)
        pred, _ , _ = model(src,tgt)
        n_class = pred.shape[-1]
        pred = pred.view(-1,n_class)# s , 단어별 예측값
        tgt = tgt.flatten() # 정답데이터는 벡터형태로

        mask = tgt > 2
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred, tgt)

        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [51]:
batch_size = 64
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# CUDA가 사용 가능한지 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


CUDA_LAUNCH_BLOCKING=1

- 토큰 인덱스가 자꾸 범위를 벗어나는 오류 해결

In [53]:
# batch = next(iter(dl))
# batch

{'src': tensor([[71, 10,  3, 11,  6, 13, 39, 31, 15, 46,  6, 24],
         [23,  3,  5, 46,  6, 24,  0,  0,  0,  0,  0,  0]]),
 'tgt': tensor([[   2, 2421,   29,   38,   61,   91,  148,    5,   34,  195,    4,    5,
             9,    6,   21,   51,   11,    4,    7,   41,   32,   29,   33,   37,
           269,   20,   76,   17,  124,    5,  194,    4,   18,   14,   15,   89,
            12,   33,    6,   27,   42,   25,   13,  411,  516,  334,    4,    5,
             9,   35,   73,   47,   11,    4,    7,   94,  483,  290,  175,  292,
           430,  338,    4,    5,   16,   12,   25,  521,   15,   96,   90,    4,
             5,    9,   35,    3],
         [   2,   31,   71,   44,    5,    9,    6,    8,   92,   24,   11,    4,
           355,   34,  118,  202,  375,   10,    7,  381,    4,   21,   50,  130,
            22,  357,  166,   46, 1078,    4,   27,   42,   25,   43,   98,   17,
            68,    4,   13,  371,   17,  198,   18,   14,  135,   44,    5,    9,
           

In [54]:
# batch['src'].view(-1)

tensor([71, 10,  3, 11,  6, 13, 39, 31, 15, 46,  6, 24, 23,  3,  5, 46,  6, 24,
         0,  0,  0,  0,  0,  0])

In [55]:
# vocab_size_src = len(vocab_review)
# vocab_size_tgt = len(vocab_reply)

In [56]:
# # train_dl에서 하나의 배치 데이터를 가져옴
# batch = next(iter(dl))

# # 배치 데이터에서 소스와 타겟을 분리
# src_batch, tgt_batch = batch

# # 인덱스 범위를 벗어나는 토큰을 찾아서 출력
# for src_idx in batch['src'].view(-1):
#     if src_idx.item() >= vocab_size_src:
#         print(f"Out of range token index: {src_idx.item()} in source")

# for tgt_idx in batch['tgt'].view(-1):
#     if tgt_idx.item() >= vocab_size_tgt:
#         print(f"Out of range token index: {tgt_idx.item()} in target")


In [57]:
# vocab_size_src, vocab_size_tgt

(3378, 3716)

#### batch 64, epoch 10
- 시작 점수 6.18 -> 끝 점수 4.22

In [58]:
reset_seeds(SEED)

model = Net(len(vocab_review), len(vocab_reply),device=device).to(device)
optimizer = torch.optim.Adam(model.parameters())

train_dt = TranslateDataset(src_data,tgt_data)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for _ in range(epochs):
    train_loss = train_loop(train_dl,model,loss_fn,optimizer,device)
    print(train_loss)

  0%|          | 0/47 [00:00<?, ?it/s]

6.187900066375732


  0%|          | 0/47 [00:00<?, ?it/s]

5.425322715272295


  0%|          | 0/47 [00:00<?, ?it/s]

...

4.457477569580078


  0%|          | 0/47 [00:00<?, ?it/s]

4.314231436303321


  0%|          | 0/47 [00:00<?, ?it/s]

4.227966891958358


In [59]:
tgt_max_len = max( len(lst) for lst in tgt_data)
tgt_max_len

287

In [74]:
@torch.no_grad()
def get_owner_reply(text,model,vocab_src,vocab_tgt,tgt_max_len,device):
    model.eval()
    src = vocab_src([ t.form for t in kiwi.tokenize(text) ])
    src = torch.tensor(src).view(1,-1).to(device)

    tgt = [2,0] # sos 토큰과 패딩토큰을 넣어서 디코더가 한번 반복 되게하기!!
    tgt = torch.tensor(tgt).view(1,-1).to(device)

    hn = None
    cn = None
    for _ in range(tgt_max_len):
        pred, hn, cn = model(src,tgt,hn,cn)
        char_no = pred[-1,-1].argmax().item()

        if char_no == 3:
            break

        print(vocab_tgt.lookup_token(char_no), end=" ")
        tgt = [char_no,0]
        tgt = torch.tensor(tgt).view(1,-1).to(device)

In [75]:
n = 1000
text = train["review"][n]
print(train["owner_reply"][n])
text

d9님, 맛있게 드셔주셔서 감사합니다! 신선하고 푸짐한 회와 다양한 스끼다시로 부족함없이 드실 수 있도록 늘 정성을 다하고 있으니 앞으로도 자주 들러주세요 감사합니당!


'대방어가 살살 녹더라구요'

In [76]:
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

감사 하 ᆸ니다 . 

In [77]:
text = '정말 맛있어요'
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

감사 하 ᆸ니다 . 

In [78]:
text = '최악이에요. 음식이 다 식어서 왔어요'
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

안녕하세요 하 ᆸ니다 . 저희 음식 을 맛있 게 들 시 었 다니 정말 감사 하 ᆸ니다 . 

In [84]:
text = '너무나도 맛있어요 사장님! 서비스로 주신 것도 감사합니다'
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 

In [85]:
text = '다신 안먹을 것 같아요'
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 

In [86]:
text = '다신 안먹을 것 같아요'
get_owner_reply(text,model,vocab_review,vocab_reply,tgt_max_len,device)

하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 하 

#### batch 2, epoch 20
- 시작점수 4.78 -> 끝 점수

In [ ]:
batch_size = 2
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 20

In [ ]:
reset_seeds(SEED)

model = Net(len(vocab_review), len(vocab_reply),device=device).to(device)
optimizer = torch.optim.Adam(model.parameters())

train_dt = TranslateDataset(src_data,tgt_data)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for _ in range(epochs):
    train_loss = train_loop(train_dl,model,loss_fn,optimizer,device)
    print(train_loss)

# -------Seq2Seq Attention 기법(미완) -------
- Attention 매커니즘 : 인코더에서 필요 정보를 디코더로 가져오는 방식
    - Q(Query) : 디코더의 t 시점 셀의 hidden state
    - K(Keys) : 인코더의 모든 시점 셀의 hidden state
    - V(Values) : 인코더 모든 시점 셀의 hidden state

-

## 어휘집 생성 (리뷰)

In [ ]:
train.head()

In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [ ]:
result = kiwi.tokenize(train["review"].astype(str))
src_data = []
for tokens in tqdm(result,total=train.shape[0]):
    tokens = [ t.form for t in tokens]
    src_data.append(tokens)

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
vocab_review = build_vocab_from_iterator(src_data,specials=["<pad>","<unk>"])
vocab_review.set_default_index(vocab_ko["<unk>"])
len(vocab_review)

In [ ]:
src_data = [ vocab_ko(tokens) for tokens in src_data ]

In [ ]:
src_data[2]

## 어휘집 생성 (답변)

## 어휘집 생성 (답변) - 형태소

In [ ]:
sos_token = "<sos>"
eos_token = "<eos>"

In [ ]:
result = kiwi.tokenize(train["owner_reply"].astype(str))

tgt_data = []
for tokens in result:
    tokens = [sos_token]+ [ t.form for t in tokens] +[eos_token]
    tgt_data.append(tokens)

In [ ]:
tgt_data[1]

['<sos>',
 '저희',
 '매장',
 '찾',
 '어',
 '주',
 '시',
 'ᆫ',
 '것',
 '도',
 '감사',
 '하',
 'ᆫ데',
 '리뷰',
 '까지',
 '..',
 '감동',
 '이',
 'ᆸ니다',
 '계속',
 '하',
 '어서',
 '더',
 '많',
 '은',
 '기대',
 '부탁드리',
 '어요',
 '부응',
 '하',
 'ᆯ',
 '수',
 '있',
 '도록',
 '최선',
 '을',
 '다',
 '하',
 '는',
 '모습',
 '을',
 '보이',
 '겠',
 '습니다',
 '다시',
 '찾',
 '어',
 '주',
 '시',
 '어도',
 '5',
 '점',
 '받',
 '을',
 '수',
 '있',
 '게',
 ',',
 '방심',
 '하',
 '지',
 '않',
 '고',
 '노력',
 '하',
 '고',
 '있',
 '겠',
 '습니다',
 '!',
 '<eos>']

In [ ]:
vocab_reply = build_vocab_from_iterator(tgt_data,specials=["<pad>","<unk>",sos_token,eos_token])
vocab_reply.set_default_index(vocab_reply["<unk>"])
len(vocab_reply)

3716

In [ ]:
vocab_en = build_vocab_from_iterator(tgt_data,specials=["<pad>","<unk>","<sos>","<eos>"])
vocab_en.set_default_index(vocab_en["<unk>"])
len(vocab_en)